In [1]:
#from copy import deepcopy

#from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, NGram, HashingTF, IDF, CountVectorizer, OneHotEncoder, StringIndexer, Word2Vec
from pyspark.sql.functions import col, udf, length, avg, lit, concat, size, array
from pyspark.sql.types import IntegerType, StringType, ArrayType
#from pyspark.mllib.classification import NaiveBayes#, NaiveBayesModel
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.sql import SparkSession, SQLContext

#import re
#from nltk.stem import SnowballStemmer
#from nltk.stem import WordNetLemmatizer
#import nltk
#nltk.download('wordnet')


In [2]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
try:
    sc.stop()
except:
    print("no sc to stop")




In [4]:
conf = SparkConf().setAppName('clickbait').setMaster('local[4]')
sc = SparkContext.getOrCreate(conf = conf)

spark = SparkSession.builder.appName('clickbait_appname').getOrCreate()
sc

<SparkContext master=local[4] appName=clickbait>

In [5]:
sc

<SparkContext master=local[4] appName=clickbait>

In [10]:
PREPROCESSED_DF_JSON_PATH="file:///home/hadoop/cb/pre_train.csv/part-00000-9f8f537d-20bc-43ef-972c-3705a1bb91aa-c000.json"
rddjson = sc.textFile(PREPROCESSED_DF_JSON_PATH)
t_df = sqlContext.read.json(rddjson)



In [ ]:
# ORIGINAL SCHEMA FROM PRREPROC SCRIPT
StructType(List(StructField(id,StringType,true),StructField(title,StringType,true),StructField(text,StringType,true),StructField(label,StringType,true),StructField(s,StringType,true),StructField(ss,ArrayType(StringType,true),true),StructField(sss,ArrayType(StringType,true),true),StructField(ssss,ArrayType(StringType,true),true),StructField(ssss_1gram,ArrayType(StringType,false),true),StructField(ssss_2gram,ArrayType(StringType,false),true),StructField(ssss_3gram,ArrayType(StringType,false),true),StructField(3gram_feats,VectorUDT,true),StructField(label_index,DoubleType,false)))

In [11]:
t_df.printSchema()

root
 |-- 3gram_feats: struct (nullable = true)
 |    |-- indices: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- size: long (nullable = true)
 |    |-- type: long (nullable = true)
 |    |-- values: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- id: string (nullable = true)
 |-- label: string (nullable = true)
 |-- label_index: double (nullable = true)
 |-- s: string (nullable = true)
 |-- ss: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- sss: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ssss: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ssss_1gram: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ssss_2gram: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ssss_3gram: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- text: string (nullab

In [12]:

train, test = t_df.randomSplit([.8,.2], seed = 999)

In [13]:
NB = NaiveBayes(modelType= "multinomial", labelCol="label_index", featuresCol ="3gram_feats")
nb_model= NB.fit(train)
nb_predictions= nb_model.transform(test)


IllegalArgumentException: 'requirement failed: Column 3gram_feats must be of type struct<type:tinyint,size:int,indices:array<int>,values:array<double>> but was actually struct<indices:array<bigint>,size:bigint,type:bigint,values:array<double>>.'

In [14]:

evaluator = MulticlassClassificationEvaluator(labelCol= "label_index", predictionCol = "prediction", 
                                              metricName="accuracy")

nb_accuracy = evaluator.evaluate(nb_predictions)

print("Accuracy : ", nb_accuracy)

NameError: name 'nb_predictions' is not defined

In [17]:
t_df.schema

StructType(List(StructField(3gram_feats,StructType(List(StructField(indices,ArrayType(LongType,true),true),StructField(size,LongType,true),StructField(type,LongType,true),StructField(values,ArrayType(DoubleType,true),true))),true),StructField(id,StringType,true),StructField(label,StringType,true),StructField(label_index,DoubleType,true),StructField(s,StringType,true),StructField(ss,ArrayType(StringType,true),true),StructField(sss,ArrayType(StringType,true),true),StructField(ssss,ArrayType(StringType,true),true),StructField(ssss_1gram,ArrayType(StringType,true),true),StructField(ssss_2gram,ArrayType(StringType,true),true),StructField(ssss_3gram,ArrayType(StringType,true),true),StructField(text,StringType,true),StructField(title,StringType,true)))